In [1]:
# === NOTES ====
# 1) If model already generated, load it.
# 2) Remeber to run ness. tokenizers

#from google.colab import drive
#drive.mount('/content/drive')

#Crach COLAB to get more RAM
'''
d=[]
while(1):
  d.append('1')
'''


import tensorflow as tf

print('Tensorflow imported')
#tf.enable_eager_execution()
#tf.executing_eagerly()
#================== Copy Rights ==================
print('='*100)
print('Copy rights: Dr. Jan P. Nees, PhD.')
print('='*100)
#================== Import libraries ==================


import os
import numpy as np
import time
import datetime
#from tensorflow import keras

print('='*100)
''' 
versionTF = tf.__version__

if versionTF != '2.0.0b1':
  !pip install tensorflow==2.0.0b1
else:
  print(versionTF)

#!pip install tensorflow-probability==0.8.0rc0
!pip install tensorflow_probability==0.8.0rc0 --user --upgrade
'''
print('='*100)

#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
#from keras.models import Model
#from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
#from keras.layers.merge import concatenate
#from keras.layers.embeddings import Embedding
#from keras.optimizers import Adam, adam
#from keras.losses import sparse_categorical_crossentropy
import matplotlib.pyplot as plt
#from keras.utils import plot_model
#from keras.models import load_model
import unicodedata
import re
import io
#from keras import Sequential
#from keras.layers import GRU, RepeatVector, TimeDistributed, Dense, LSTM, Input
#from keras.losses import sparse_categorical_crossentropy
#from keras.callbacks import TensorBoard
from time import time
import matplotlib.ticker as ticker
#from nltk.tokenize import sent_tokenize
#from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import collections



FLASK_DEBUG=0

print ('Lib finished importing')

#================== Methods ==================
#---------Tokenizer Method -----------------------------------
# ------ Main issues: Not sure if it does it correctly to DK and DE sentences. --------

#================== ======================================== ==================
#english_reader = load_data('europarl-v8.en')
#danish_reader = load_data('europarl-v8.da')

print('Dataset Loaded')

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    #w = re.sub(r"[å]+", "aa", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

#----------------------------------------------------------------------
def load_data_v2(path):
    
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    #w = preprocess_sentence(line) for line in lines
    #return w
    w = [preprocess_sentence(line)  for line in lines[8000:16000]] #<----------- How many lines of sentences
        #print(w)
            #data = preprocess_sentence(lines)
            #preprocess_sentence(data) 

    return w
#----------------------------------------------------------------------
print("""
Models has been loaded
Methods has been loaded
""")

# ==== Colab Path ====
#english_reader = load_data_v2('drive/My Drive/txt/europarl-v7.da-en.en')
#danish_reader = load_data_v2('drive/My Drive/txt/europarl-v7.da-en.da')

# === EURO PARL ===
# ==== Local Path ====
english_reader = load_data_v2('europarl-v7.da-en.en')
danish_reader = load_data_v2('europarl-v7.da-en.da')
print('Load finished')

# === ANKI ====
#english_reader = load_data_v2('UK.txt')
#danish_reader = load_data_v2('DK.txt')

#=============================================================================
#================= TEXT ANALYSIS HERE ========================================
#=============================================================================

#------------- DATA processing-----------------
print('------- Data processing -------------')

def max_length(tensor):
    return max(len(t) for t in tensor)

def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')

  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, lang_tokenizer

print('tokenizer loaded')

target_tensor, targ_lang = tokenize(english_reader)
input_tensor, inp_lang = tokenize(danish_reader)

print("tensor is (rows, sentence length padded)")
print('target_tensor.shape: ', target_tensor.shape) # <--- (8000, 143)
print('input_tensor.shape: ', input_tensor.shape)   # <--- (8000, 146)

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)) # 6400 6400 1600 1600 

def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))
    
print('Model loaded')

print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[1])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[1])

Tensorflow imported
Copy rights: Dr. Jan P. Nees, PhD.
Lib finished importing
Dataset Loaded

Models has been loaded
Methods has been loaded

Load finished
------- Data processing -------------
tokenizer loaded
tensor is (rows, sentence length padded)
target_tensor.shape:  (8000, 143)
input_tensor.shape:  (8000, 146)
6400 6400 1600 1600
Model loaded
Input Language; index to word mapping
3 ----> <start>
21 ----> med
92 ----> hensyn
12 ----> til
8 ----> det
146 ----> andet
82 ----> forslag
430 ----> forstaar
34 ----> kommissionen
1542 ----> oensket
22 ----> om
1 ----> ,
5 ----> at
8 ----> det
27 ----> skal
36 ----> vaere
144 ----> klart
1 ----> ,
64 ----> hvad
15 ----> der
13007 ----> forstaas
40 ----> ved
13 ----> en
4994 ----> selvstaendig
2 ----> .
34 ----> kommissionen
156 ----> kommer
21 ----> med
13 ----> en
407 ----> loesning
11 ----> af
30 ----> dette
391 ----> problem
7 ----> i
8 ----> det
1729 ----> aendrede
82 ----> forslag
2 ----> .
4 ----> <end>

Target Language; index to wo

In [2]:

#===================================
# Test Normalization vs Embedding??
#===================================
print(input_tensor_train.shape)
print("""
      Shape (19,18,5)
      axis 0 = 19
      axis 1 = 18
      axis 2 = 5
      axis -1 = last axis
      """
      )
''' 
input_tensor_train_norm = tf.keras.utils.normalize(input_tensor_train, axis=-1)
target_tensor_train_norm = tf.keras.utils.normalize(target_tensor_train, axis=-1)
print(input_tensor_train[0])
print(input_tensor_train_norm[0])
print(np.max(input_tensor_train_norm[0]))
'''

(6400, 146)

      Shape (19,18,5)
      axis 0 = 19
      axis 1 = 18
      axis 2 = 5
      axis -1 = last axis
      


' \ninput_tensor_train_norm = tf.keras.utils.normalize(input_tensor_train, axis=-1)\ntarget_tensor_train_norm = tf.keras.utils.normalize(target_tensor_train, axis=-1)\nprint(input_tensor_train[0])\nprint(input_tensor_train_norm[0])\nprint(np.max(input_tensor_train_norm[0]))\n'

In [3]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[3])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[3])
print('='*100)

#===============================================================================
#===============================================================================
print('len(target_tensor_train): ', len(target_tensor_train))
print('len(target_tensor_val): ', len(target_tensor_val))

print('target_tensor.shape[1]: ', target_tensor.shape[1])
print('input_tensor.shape[1]: ', input_tensor.shape[1])



#n_input = len(target_tensor_val)    #<--- 6400
#n_output = len(target_tensor_train) #<--- 1600

n_input = input_tensor.shape[1] # (8000, 143)
n_output = target_tensor.shape[1]   # (8000, 146)
#===============================================================================

n_units = 4096 #what should this be?

n_steps = target_tensor.shape[0] # ---- 8000

vocab_dk = len(inp_lang.word_index)+1
vocab_uk = len(targ_lang.word_index)+1 

embedding_dim = 256
BATCH_SIZE = 64
hidden = 0

print('Vocab size DK: ', vocab_dk)
print('Vocab size UK: ', vocab_uk)

Input Language; index to word mapping
3 ----> <start>
8 ----> det
9 ----> er
165 ----> dog
120 ----> helt
144 ----> klart
1 ----> ,
5 ----> at
49 ----> disse
3036 ----> stoetteordninger
80 ----> boer
2206 ----> kontrolleres
1 ----> ,
6 ----> og
8 ----> det
9 ----> er
13 ----> en
394 ----> opgave
1 ----> ,
15 ----> der
3708 ----> paahviler
39 ----> europa
34 ----> kommissionen
2 ----> .
4 ----> <end>

Target Language; index to word mapping
3 ----> <start>
307 ----> clearly
544 ----> though
2 ----> ,
18 ----> it
42 ----> must
17 ----> be
2507 ----> monitored
2 ----> ,
10 ----> a
651 ----> task
20 ----> which
1535 ----> falls
7 ----> to
1 ----> the
28 ----> european
29 ----> commission
5 ----> .
4 ----> <end>
len(target_tensor_train):  6400
len(target_tensor_val):  1600
target_tensor.shape[1]:  143
input_tensor.shape[1]:  146
Vocab size DK:  14407
Vocab size UK:  9306


In [4]:
#===============================================================================
#input_dim: This is the size of the vocabulary in the text data. 
# For example, if your data is integer encoded to values between 0-10, then 
# the size of the vocabulary would be 11 words.

#output_dim: This is the size of the vector space in which words will be 
# embedded. It defines the size of the output vectors from this layer for 
# each word. For example, it could be 32 or 100 or even larger.

#input_length: This is the length of input sequences, as you would define for 
# any input layer of a Keras model. For example, if all of your input 
# documents are comprised of 1000 words, this would be 1000.
# example: model.add(Embedding(vocab_size, 8, input_length=max_length))
#===============================================================================

In [5]:
import time
start = time.time()

#===============================================================================
encoder_inputs = tf.keras.layers.Input(shape=(n_input), name='Encoder_Input')

encod_embed = tf.keras.layers.Embedding(output_dim=n_input, input_dim=vocab_dk, input_length=vocab_dk, name='Encod_Embed') # <- is this right??
enc_lstm = tf.keras.layers.LSTM(n_units, return_state = True, return_sequences=True, name='LSTM_1')
#===============================================================================
encd_embed_out = encod_embed(encoder_inputs)
enc_out, enc_h, enc_c = enc_lstm(encd_embed_out)
enc_states = [enc_h, enc_c]

decoder_inputs = tf.keras.layers.Input(shape=(n_output), name='Decoder_Input')

decod_embed = tf.keras.layers.Embedding(output_dim=n_output, input_dim=vocab_uk, name='Decod_Embed')
dec_lstm = tf.keras.layers.LSTM(n_units, return_state = True, return_sequences=True, name='Decoder_LSTM') # Build the LSTM layer
decoder_output = tf.keras.layers.Dense(n_output, activation='softmax', name='Decoder_Dense')

decod_embed_out = decod_embed(decoder_inputs)
dec_x, dec_h, dec_c = dec_lstm(decod_embed_out, initial_state=enc_states) # Feed to the LSTM layer
dec_states = [dec_h, dec_c] # Seperate out the states
    
decoder_out_dense = decoder_output(dec_x)
#===============================================================================
model_test_NoClass = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_out_dense)
#===============================================================================
#           Compile
#===============================================================================
model_test_NoClass.compile(optimizer='adam', 
                           loss='categorical_crossentropy', 
                           metrics=['accuracy']) 
                                # <- WORKS!!!!
#===============================================================================
model_test_NoClass.summary()
print('Time taken sec:\n', ((time.time()-start)*60))
#tf.keras.utils.plot_model(model_test_NoClass, show_shapes=True)
#tf.keras.utils.plot_model(model_test_NoClass, show_shapes=True,  to_file='Enc_Decoder_NoClass_v2.png')

#===============================================================================
#tensor board test
#!rm -rf ./logs/

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_Input (InputLayer)      [(None, 146)]        0                                            
__________________________________________________________________________________________________
Decoder_Input (InputLayer)      [(None, 143)]        0                                            
__________________________________________________________________________________________________
Encod_Embed (Embedding)         (None, 146, 146)     2103422     Encoder_Input[0][0]              
__________________________________________________________________________________________________
Decod_Embed (Embedding)         (None, 143, 143)     1330758     Decoder_Input[0][0]              
______________________________________________________________________________________________

In [6]:
#%load_ext tensorboard

log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

print("Tensorboard setup finished")

# tensorboard does not work with tensorflow.

Tensorboard setup finished


In [7]:
#===============================================================================
batchsize = 60
print(target_tensor.shape)
print(input_tensor.shape)
print(input_tensor_val.shape)
print(target_tensor_val.shape)
inp_tensor_shape_val = input_tensor_val.shape[0]
print('Does the shape and batch size work?: ', input_tensor.shape[0] / batchsize)
#===============================================================================
emptyContainer2 = tf.zeros([inp_tensor_shape_val, n_output, 1], dtype=tf.dtypes.float64)
print(emptyContainer2.shape) # <--- shape = (8000, 143, 1)

print('Starting Evaluation modelling.')
print('Eval the model apparently can only handle verbose =1')
print('verbose 0 = silence, 1 = progress bar, 2 = per epocch')

score = model_test_NoClass.evaluate([input_tensor_val, target_tensor_val], 
                                    emptyContainer2,  
                                    batch_size=batchsize, 
                                    verbose=1,
                                    callbacks=[tensorboard_callback]
                                   )
print('Eval model finished')
#===============================================================================
print("%s: %.2f%%" % (model_test_NoClass.metrics_names[1], score[1]*100))
print('Metrics: ', model_test_NoClass.metrics_names)
print('Input names: ', model_test_NoClass.input_names)
print('Output names: ', model_test_NoClass.output_names)
print('='*100)
#===============================================================================


(8000, 143)
(8000, 146)
(1600, 146)
(1600, 143)
Does the shape and batch size work?:  133.33333333333334
(1600, 143, 1)
Starting Evaluation modelling.
Eval the model apparently can only handle verbose =1
verbose 0 = silence, 1 = progress bar, 2 = per epocch
1600/1 [==============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Eval model finished
accuracy: 0.77%
Metrics:  ['loss', 'accuracy']
Input names:  ['Encoder_Input', 'Decoder_Input']
Output names:  ['Decoder_Dense']


In [ ]:
# log_dir = "logs\\fit\\"

#!tensorboard --logdir log_dir
#%tensorboard --logdir {log_dir}

In [ ]:
# ===============================================================================
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val  = train_test_split(input_tensor, target_tensor, test_size=0.2)
# uk_training_reshape -> is: input_tensor_train
# dk_training_reshape -> is: target_tensor_train
# uk_test_reshape -> is: input_tensor_val
# dk_test_reshape -> is: target_tensor_val
# ==============================================================================
# Current error - needs to be reshaped into 3ndim. How can Google avoid this? Embedding layer??



start = time.time()

# ========== FIT MODEL =========================================================
es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min')

start = time.time()

con_value = input_tensor_train.shape[0]
emptyContainer2 = tf.zeros([con_value, n_output, 1], dtype=tf.dtypes.float32, name=None)

model_test_NoClass.fit([input_tensor_train, target_tensor_train], 
                       emptyContainer2, 
                       batch_size = 30, 
                       epochs=10, 
                       verbose=1, # Epochs have to be set to minimum 10
                       validation_split=0.3,
                       callbacks=[tensorboard_callback]
                      ) # Uses Train data
#===============================================================================
#===============================================================================
plt.plot(model_test_NoClass.history.history['loss'])
plt.plot(model_test_NoClass.history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
#===============================================================================
print('Time taken sec:\n', ((time.time()-start)*60))
#===============================================================================
# save model and architecture to single file
model_test_NoClass.save("Google_Rebuild_FitModel_v8000_ep3_v2.h5")
print("Saved model to disk")
#===============================================================================



In [ ]:
#val_loss, val_acc = model_test_NoClass.evaluate([input_tensor_val, target_tensor_val], emptyContainer2)
#print(val_loss)
#print(val_acc)

In [ ]:
emptyContainer3 = tf.zeros([inp_tensor_shape_val, n_output, 1], dtype=tf.dtypes.float64)
score = model_test_NoClass.evaluate([input_tensor_val, target_tensor_val], emptyContainer3, verbose=1)
print('Score after training: \n', model_test_NoClass.metrics_names, score)
print('='*200)
#print('optimizer: ', optimizer)
print('Score after training: \n', model_test_NoClass.metrics_names, score)
print("%s: %.2f%%" % (model_test_NoClass.metrics_names[1], score[1]*100))

In [ ]:
# How to see if the restore model works???
# See that now its empty, and after below restore, its been populated?

print()


In [ ]:
#========================================
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! LOAD ME
#========================================
# Remeber to load basic elements, then run this ...
# Then I don t need to train again .... Again the 28 hrs model could be run at my own system.
#
#========================================
# OLD model
#model_restored = tf.keras.models.load_model('drive/My Drive/txt/Google_Rebuild_FitModel.h5')
#emptyContainer4 = tf.zeros([n_steps, n_output, 1], dtype=tf.dtypes.float32, name=None) 
# NEW model
model_restored = tf.keras.models.load_model('Google_Rebuild_FitModel_v8000_ep3.h5')
# summarize model.
model_restored.summary()
#========================================
#score = model_restored.evaluate([input_tensor_train, target_tensor_train], emptyContainer2, verbose=1)
#print("%s: %.2f%%" % (model_restored.metrics_names[1], score[1]*100))
tf.keras.utils.plot_model(model_test_NoClass, show_shapes=True)

In [ ]:
#===============================================================================
#===============================================================================
# Inference models for testing
#===============================================================================
# This is a strange piece of code, the fitting has already been run.
# So what is the point with the below?
# train_test.fit([input_tensor_train_res, target_tensor_train_res], emptyContainer2, epochs=2, verbose=1, callbacks=[es])
#===============================================================================

#===============================================================================
# REUSE CODE
#===============================================================================

print('len(target_tensor_train): ', len(target_tensor_train))
print('len(target_tensor_val): ', len(target_tensor_val))

print('target_tensor.shape[1]: ', target_tensor.shape[1])
print('input_tensor.shape[1]: ', input_tensor.shape[1])

import time
start = time.time()

#n_input = len(target_tensor_val)    #<--- 6400
#n_output = len(target_tensor_train) #<--- 1600

n_input = input_tensor.shape[1] # (8000, 143)
n_output = target_tensor.shape[1]   # (8000, 146)

n_units = 1024
optimizer = 'adam'
loss_unit = 'mean_squared_error'
n_steps = target_tensor.shape[0] # ---- 8000

vocab_dk = len(inp_lang.word_index)
vocab_uk = len(targ_lang.word_index) 

print('Vocab size DK: ', vocab_dk)
print('Vocab size DK: ', vocab_uk)
print('N_output: ', n_output)

#===============================================================================
#===============================================================================
# EXPERIMENTAL CODE
#===============================================================================
#===============================================================================
#model_restored.
#===============================================================================
print('n_units: ', n_units)

#===============================================================================
# Encoder inference model
encoder_model_inf = tf.keras.Model(encoder_inputs, enc_states, name='Encoder Model') # works even with Embedding
print('='*100)
encoder_model_inf.summary()

# Embedding layer?
#===============================================================================
# Decoder inference model
#shape should be ???
decoder_state_input_h = tf.keras.layers.Input(shape=(n_units,)) # Empty Container (1024)
decoder_state_input_c = tf.keras.layers.Input(shape=(n_units,)) # Empty Container (1024)
decoder_input_states = [decoder_state_input_h, decoder_state_input_c] # Connects the two containers: shape=(None, 1024)
final_dex = decod_embed(decoder_inputs)
print(decoder_input_states)
# Embedding layer
#Embed_inf = decoder_inputs

#===================================
# Below takes DEC_LSTM and feed decoder_inputs and set the empty containers.
decoder_out2, decoder_h2, decoder_c2 = dec_lstm(final_dex, 
                                                 initial_state=decoder_input_states) # why does this fail?, Shape (143, None) <- where?

decoder_states2 = [decoder_h2, decoder_c2]
decoder_outputs2 = decoder_output(decoder_out2)
#===============================================================================
# Decoder inference model
decoder_model_inf = tf.keras.Model(inputs=[decoder_inputs] + decoder_input_states, # <--------- AS ALWAYS IS THE NAMING ISSUES
                          outputs=[decoder_outputs2] + decoder_states2 )
#===============================================================================

print('='*100)
decoder_model_inf.summary()
print('='*100)

tf.keras.utils.plot_model(encoder_model_inf, show_shapes=True, to_file='Infer_Encoder_NoClass.png')
tf.keras.utils.plot_model(decoder_model_inf, show_shapes=True,  to_file='Infer_Decoder_NoClass.png')

In [ ]:

#===============================================================================
#===============================================================================
#========== TESTING PREDICTION ================ ================================
# ========= DK 2 UK Translation =============== ================================
# We want to feed it DK sentence and let it translation into UK
# What would the final version be for the research paper???? UK to Many???
#===============================================================================
# So far so good ... now how to Predict and get out the stuff that I want?

max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

print('='*100)
text_testing_dk = 'Det har du rigtig godt af'

#add <start> x <end>
sentence = preprocess_sentence(text_testing_dk)
print('sentence: ', sentence)
print('='*100)

#to index:  [3, 457, 22, 3560, 4]
inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
print('to index: ', inputs) 
print('='*100)
convert(inp_lang, inputs)

# create the Matrix
inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
inputs = tf.convert_to_tensor(inputs)
print (inputs)

print('Starting to send to NN')
print('='*100)

#units = 1024
#hidden = [tf.zeros((1, units))]
#enc_out, enc_hidden = encoder_model_inf(inputs, hidden)

#dec_hidden = enc_hidden
#print(dec_hidden)
#===============================================================================
# Need to make it into 3D.
#===============================================================================
print('inputs.shape: ', inputs.shape) # (1,146)
inputs_re = tf.reshape(inputs,(1,max_length_inp))
#inputs_re = inputs.reshape(1,max_length_targ,1) #  <---- should this shape match something?
print('inputs_re.shape: ', inputs_re.shape)
#print('the inputs looks like: \n', inputs_re)
print('='*100)



In [ ]:
print(target_tensor.shape[1])
print(input_tensor.shape[1])
print(max_length_targ)

In [ ]:
#target_tensor.shape[1]:  12
#input_tensor.shape[1]:  13

print('targ lang end: ', targ_lang.index_word[4] == '<end>')

states_val_v2 = encoder_model_inf.predict(inputs_re) # inputs shape (146,1)
print(states_val_v2)
print('='*100)
target_seq = np.zeros((1, max_length_targ)) # shape (1, 143) <--- WORKS
target_seq[0,0] = targ_lang.word_index['<start>']
print(target_seq.transpose)
print('='*100)
print('target seq: ', target_seq.shape)


stop_con = False
decoded_sen = ''
while not stop_con:
   prediction, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val_v2) # <- do this 143 times, why?
   
   sampled_token_index = np.argmax(prediction[0,-1,:])
   sampled_char = targ_lang.index_word[sampled_token_index]
   decoded_sen += ' ' + sampled_char
   print(decoded_sen)
   print('='*100)

   if(len(decoded_sen) > max_length_targ):
     stop_con = True
   target_seq = np.zeros((1,12)) # <--- WORKS, well to some degree...
   target_seq[0,0] = sampled_token_index
   #states_val_v2 = [h,c]

print(decoded_sen)
print('='*100)


In [ ]:
print('targ lang end: ', targ_lang.index_word[4] == '<end>')
import tensorflow_probability as tfp

''' 
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


states_val_v2 = encoder_model_inf.predict(inputs)
print(states_val_v2)
target_seq = np.zeros((143,1))
target_seq[0,0] = targ_lang.word_index['<start>']
print(target_seq.transpose)

stop_con = False
decoded_sen = ''
while not stop_con:
   prediction, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val_v2) # <- do this 143 times, why?

   sampled_token_index = np.argmax(prediction[0,-1,:])
   sampled_char = targ_lang.index_word[sampled_token_index]
   decoded_sen += ' ' + sampled_char
   print(decoded_sen)



   if(len(decoded_sen) > 143):
     stop_con = True
   target_seq = np.zeros((1,1))
   target_seq[0,0] = sampled_token_index
   #states_val_v2 = [h,c]

print(decoded_sen)


#for t in range(1, max_length_targ): # does this mean 1-146 <- yes: "range(min_value, max_value)"
#===============================================================================
# So here we take the statevalues from the Dncoder, and try to predict these in the Decoder
# We then take the prediction/decoder out - and feed it through a argmax to find the max values.

#    prediction, decoder_h, decoder_c = decoder_model_inf.predict(x=[decoder_inp] + states_val) # <- do this 143 times, why?


  #print('prediction.tolist: \n', prediction)
  #print(t, ':', 'prediction[1:1,:]: \n', prediction[:,:t])
  #print('prediction shap: \n', prediction.shape) # (1, 143, 143)
  #print('='*100)

  #predict_id = tf.math.argmax(input=prediction[:,:t], axis=1)
  #predict_id = np.argmax(prediction[:,:t])
  #print(t, ':', predict_id)
'''

'''  
  #predict_id = tfp.distributions.Multinomial(tf.math.exp(prediction), probs=prediction)[0][0]

  #
  print('predict_id = tf.argmax: \n', predict_id)
  #result += targ_lang.index_word[predict_id] + ' '

  if targ_lang.index_word[predict_id] == '<end>':
    print('result: ', result)
    print('finsihed')

  print('result: ', result)
  #prediction_id = tf.exp(prediction)[0][0].numpy()
  #print(prediction_id)
  
  #print("prediction: \n", prediction.shape) # <----- (1,143,143) ---- there are different numbers inside it, I guess we need a loop to get, WHY inside loop???
  #print('np.argmax(prediction): \n', np.argmax(prediction)) #<--- find the highest number in Matrix.
  #print('predict_id = tf.argmax: \n', tf.argmax(decoder_out[0:142,:]).numpy()) # Spit out an array with many values, would make sense. [:] <- [from:to]
  #for t in range(1,2):
   # print('prediction[:,:-1]: \n', prediction[:-1,:])


  #max_val_index = np.argmax(prediction[0,-1,:]) # <- what does this Matrix slicing do??? [R,C,]
  #sampled_fra_char = targ_lang.index_word[max_val_index]

  #while t < 2:
   # print(t, ' ', decoder_out)

  #print(sampled_fra_char)

  #translated_sent += sampled_fra_char


 
  #print('decoder_out.shape: ', decoder_out.shape)

  #print('='*100)
  #print('Decoder out in 0: \n', tf.argmax(decoder_out[0]))
  #predict_id = tf.argmax(decoder_out[0,-1,:]).numpy() # Spit out an array with many values, would make sense.
  #print('='*100)
  
  #print(t, ' prediction: \n', predict_id)
  #print('='*100)
  
  #translated_sent += predict_id

  #predict_id = np.argmax(decoder_out[0]) # Spit out one value - does not make sense
  #translated_sent += targ_lang.index_word[predict_id] + ' '
  
  #if targ_lang.index_word[predict_id] == '<end>':
   # print('finsihed')


#print('prediction: \n', predict_id)
#print('result: \n', translated_sent)
'''

''' 
print(max_val_index)
print('*'*100)
print(tf.math.argmax(input=decoder_out))
print('='*100)
print('decoder shape: \n', decoder_out.shape)
'''

# Then we pass this max_val into an ARGMAX to find the max value.
#===============================================================================
#max_val = np.argmax(decoder_out[0,-1,:])
#sampled_char = targ_lang.index_word[max_val] # <---- or word_index ??? 


In [ ]:
# function for plotting the attention weights
def plot_attention(sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()


# We first need to find a connection in the Encoder

# Test
#(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_out_dense)

#inputs_reshape = inputs.reshape(inputs.shape[0],inputs.shape[1],1)

#model_restored.predict([inputs])

In [ ]:
#========== TESTING PREDICTION ================ ================================
# ========= DK 2 UK Translation =============== ================================
# We want to feed it DK sentence and let it translation into UK
# What would the final version be for the research paper???? UK to Many???
#===============================================================================
# So far so good ... now how to Predict and get out the stuff that I want?

#print('Word list UK: ', english_fit2text.word_index)
#print('Word list DK: ', danish_fit2text.word_index)

#targ_lang <- UK
#inp_lang <- DK

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

print('='*100)

print(max_length_inp)

print('='*100)
text_testing_dk = 'der var stilhed'
#add <start> and <end>
sentence = preprocess_sentence(text_testing_dk)
print(sentence)
inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
inputs = tf.convert_to_tensor(inputs)

print(inputs)
print('='*100)


print('='*100)
testing_txt2seq_dk = inp_lang.texts_to_sequences(text_testing_dk)
print('testing_txt2seq_dk: ', testing_txt2seq_dk)
#print('testing_txt2seq dk: ', np.transpose(testing_txt2seq_dk))

print('='*100)
# What do I want to prove here - that I can go back?
testing_seq2txt = inp_lang.sequences_to_texts(testing_txt2seq_dk)
print('testing_seq2txt dk: ', testing_seq2txt)

print('='*100)

#===============================================================================
#testing_txt2seq_pad_dk = pad(testing_txt2seq_dk, max_sequence_length)
#testing_txt2seq_pad_uk = pad(testing_txt2seq_uk, max_sequence_length)

#print('testing_txt2seq_pad dk: ', testing_txt2seq_pad_dk)
#print('testing_txt2seq_pad uk: ', testing_txt2seq_pad_uk)
print('='*100)
#print('Shape dk: ', testing_txt2seq_pad_dk.shape)
#print('Shape uk: ', testing_txt2seq_pad_uk.shape)
#testing_txt2seq_reshaped = testing_txt2seq_pad.reshape((-1, testing_txt2seq_pad[-1]))
print('='*100)

#testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(*testing_txt2seq_pad_dk.shape, 1)
#testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(*testing_txt2seq_pad_uk.shape, 1)

max_sequence_length = input_tensor.shape[1]
testing_txt2seq_pad_dk = pad(testing_txt2seq_dk, max_sequence_length)

testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(2,max_sequence_length,1)
testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(2,max_sequence_length,1)


print('testing_txt2seq_reshaped: ', testing_txt2seq_reshaped_dk.shape)
print('testing_txt2seq_reshaped_uk: ', testing_txt2seq_reshaped_uk.shape)
print('='*100)

#======================================================================================
dims2 = train_test.predict(testing_txt2seq_dk) #<---- What is the input UK , DK ???
print(dims2)